# Compare different path planning algorithms for information collection in the context of a static environment
* Initialize a static environment
* Create a world with one robot, with different policies
* Run it for a certain amount of time. Collect the estimation error. 
* Visualize the different results

In [ ]:
%matplotlib inline
import LoggingSetup
import logging
import itertools
import numpy as np
import ast
import math
import random
import pathlib
import itertools
from functools import partial
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from Environment import Environment, PollutionModelEnvironment, EpidemicSpreadEnvironment
from InformationModel import InformationModel, ScalarFieldInformationModel_stored_observation
from Robot import Robot
from Policy import GoToLocationPolicy, FollowPathPolicy, RandomWaypointPolicy, \
    AbstractWaypointPolicy, generate_lawnmower, InformationGreedyPolicy, generate_lawnmower
from World import World, WorldFactory
from VisualWorld import VisualWorld

In [ ]:
# the colection of all the results for all experiments
results = {}
width = 10
height = 10
seed = 1
maxtime = 30

def run_experiment(policy_desc, visual = False):
    logging.getLogger().info(f"Running experiment witt {policy_desc}")
    experiment = {}
    experiment["mse"] = []
    world = WorldFactory.generate_world_pretrained_static_pollution(width=width, height=height, seed=seed)
    robot = Robot("Robur", 4, 8, 0, env=world.env, im=world.im)
    robot.policy = eval(policy_desc)
    world.add_robot(robot)
    if visual:
        visual_world = VisualWorld(world)
    for t in range(maxtime):
        world.enact_policy(1)
        world.proceed(1)
        mse = mean_squared_error(world.env.value, world.im.value)
        experiment["mse"].append(mse)
        if visual:
            visual_world.update_visualization()
    return experiment
 
policy_desc = "InformationGreedyPolicy(robot, 1, span=5)";
results["Greedy-5"] = run_experiment(policy_desc, visual = True)    
policy_desc = "InformationGreedyPolicy(robot, 1, span=10)";
results["Greedy-10"] = run_experiment(policy_desc, visual = True)   
policy_desc = "RandomWaypointPolicy(None, robot, 1, [0,0], [width-1, height-1], seed=1)";
results["RandomWaypoint"] = run_experiment(policy_desc)
policy_desc = "RandomWaypointPolicy(None, robot, 1, [0,0], [width-4, height-4], seed=1)";
results["RandomWaypoint-reduced"] = run_experiment(policy_desc)
policy_desc = "RandomWaypointPolicy(None, robot, 1, [0,0], [width, height], seed=1)";
results["RandomWaypoint-extended"] = run_experiment(policy_desc)


In [ ]:
fix, ax = plt.subplots()
for name in results:
    experiment = results[name]
    ax.plot(experiment["mse"], label=name)
ax.legend()